# Create CPI by category dataset

This script retrieves the CPI data by top-level category and region. It processes the data into a csv file to be used in other analysis. 

Kim J. Ruhl // July 13, 2022 // ruhl2@wisc.edu

In [1]:
import requests
import json
import pandas as pd

# https://www.bls.gov/regions/midwest/data/xg-tables/ro5xg01.htm

var = {'CUUR0200SA0':'midwest', 'CUURS200SA0':'midwestA', 'CUURN200SA0':'midwestBC', 'CUUR0360SA0':'midwestENC',
      'CUUR0200SAA':'apparel', 'CUUR0200SAE':'education', 'CUUR0200SAF':'foodbev', 'CUUR0200SAH':'housing',
      'CUUR0200SAM':'medical', 'CUUR0200SAR':'recreation', 'CUUR0200SAT':'transportation'}

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": list(var),"startyear":"2019", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

In [2]:
def extract_df(s):
    temp = pd.DataFrame(s['data'])[['year', 'period', 'value']]
    temp = temp.rename(columns={'value':var[s['seriesID']]})
    temp['date'] = pd.to_datetime(temp['year'] + '-' + temp['period'].str.slice(1,))
    temp = temp.drop(['year', 'period'], axis=1).set_index('date')
    return temp

In [3]:
i = iter(json_data['Results']['series'])
data = extract_df( next(i) )

for s in i:
    data = data.join( extract_df(s))

data.tail()

,midwest,midwestA,midwestBC,midwestENC,apparel,education,foodbev,housing,medical,recreation,transportation
date,,,,,,,,,,,
2019-05-01,238.219,238.926,152.284,102.455,114.920,137.108,247.517,232.790,492.063,121.203,207.945
2019-04-01,237.510,237.576,152.093,102.541,115.838,137.404,246.391,231.999,487.594,120.985,207.665
2019-03-01,236.793,237.414,151.405,102.336,117.625,137.446,246.618,231.701,491.086,121.194,203.171
2019-02-01,235.444,236.109,150.523,101.712,118.579,137.823,247.108,230.869,491.532,121.032,197.175
2019-01-01,233.837,234.773,149.383,101.116,114.788,137.834,245.739,230.194,492.369,121.229,192.640


In [4]:
data.to_csv('cpis.csv')